In [ ]:
# Copyright (C) 2016-2018 by Jakob J. Kolb at Potsdam Institute for Climate
# Impact Research
#
# Contact: kolb@pik-potsdam.de
# License: GNU AGPL Version 3
%matplotlib inline
#import PyDSTool as pdt
import numpy as np
import networkx as nx
from random import shuffle
import sympy as sp
import pandas as pd
import matplotlib.pyplot as plt
from pydivest.macro_model.integrate_equations_aggregate import IntegrateEquationsAggregate as MacroModel
from pydivest.micro_model.divestmentcore import DivestmentCore as MicroModel
from pydivest.default_params import ExperimentDefaults 

from pymofa.experiment_handling import even_time_series_spacing


In [ ]:
# investment_decisions:

nopinions = [50, 50]

# Parameters:

phi, b_d = 0.5, 3.

defaults = ExperimentDefaults(phi=0.5,
                              b_d=4.,
                              L=100.,
                              R_depletion=False)

input_parameters = defaults.input_params

# investment_decisions
opinions = []
for i, n in enumerate(nopinions):
    opinions.append(np.full((n), i, dtype='I'))
opinions = [item for sublist in opinions for item in sublist]
shuffle(opinions)

# network:
N = sum(nopinions)
p = .2

while True:
    net = nx.erdos_renyi_graph(N, p)
    if len(list(net)) > 1:
        break
adjacency_matrix = nx.adj_matrix(net).toarray()

# investment
clean_investment = np.ones(N)
dirty_investment = np.ones(N)

init_conditions = (adjacency_matrix, opinions,
                   clean_investment, dirty_investment)
print(input_parameters)

In [ ]:
m = MicroModel(*init_conditions, **input_parameters)

t_m = 0
xis = []
t_len = 50
data_points = 21
xi_max = .2
xi_min=.1
for xi in np.linspace(xi_min, xi_max, data_points):
    print(xi)
    t_m += t_len
    xis += [xi]*t_len
    m.xi = xi
    m.set_parameters()
    m.run(t_max=t_m)

trj1 = even_time_series_spacing(m.get_aggregate_trajectory(), t_len*data_points)
trj1['xi'] = xis
xis = []

m.init_aggregate_trajectory()

for xi in np.linspace(xi_max, xi_min, data_points):
    print(xi)
    t_m += t_len
    xis += [xi]*t_len
    m.xi = xi
    m.set_parameters()
    m.run(t_max=t_m)
    
trj2 = even_time_series_spacing(m.get_aggregate_trajectory(), t_len*data_points)
trj2['xi'] = xis

mi1_xi_means = trj1.groupby('xi').mean()
mi1_xi_std = trj1.groupby('xi').std()

mi2_xi_means = trj2.groupby('xi').mean()
mi2_xi_std = trj2.groupby('xi').std()

fig, axes = plt.subplots(ncols=3)
fig.set_figwidth(15)

trj1[['C']].plot(ax=axes[0])
trj1[['x', 'z']].plot(ax=axes[1])

trj2[['C']].plot(ax=axes[0])
trj2[['x', 'z']].plot(ax=axes[1])

axb = axes[2].twinx()
mi1_xi_means[['x']].plot(ax=axb, style='go')
mi2_xi_means[['x']].plot(ax=axb, style='gx')

In [ ]:
fig, axes = plt.subplots(ncols=3)
fig.set_figwidth(15)

trj1[['C']].plot(ax=axes[0])
trj1[['x', 'z']].plot(ax=axes[1])

trj2[['C']].plot(ax=axes[0])
trj2[['x', 'z']].plot(ax=axes[1])

axb = axes[2].twinx()
mi1_xi_means[['x']].plot(ax=axb, style='go')
mi2_xi_means[['x']].plot(ax=axb, style='gx')

In [ ]:
m = MacroModel(*init_conditions, **input_parameters)

t_m = 0
xis = []
t_len = 30
data_points = 21
xi_max = .175
for xi in np.linspace(0.12, xi_max, data_points):
    print(xi)
    t_m += t_len
    xis += [xi]*t_len
    m.p_xi = xi
    m.set_parameters()
    m.run(t_max=t_m)

trj = even_time_series_spacing(m.get_aggregate_trajectory(), t_len*data_points)
trj['xi'] = xis

ma_xi_means = trj.groupby('xi').mean()
ma_xi_std = trj.groupby('xi').std()

In [ ]:
path = "../output_data/X7/mean_trj.h5"
path_err = "../output_data/X7/std_trj.h5"

with pd.HDFStore(path) as store:
    data1 = store.select('dat_0')
    data2 = store.select('dat_1')
    
with pd.HDFStore(path_err) as store:
    data1_err = store.select('dat_0')
    data2_err = store.select('dat_1')

# print(len(data1.index.levels[5]))
# print(len(data2.index.levels[5]))

# t_n = 100
# xis1 = []
# xis2 = []
# data_points = 51
# xi_min = 0.1
# xi_max = 0.2
# for xi1, xi2 in zip(np.linspace(xi_min, xi_max, data_points), np.linspace(xi_max, xi_min, data_points)):
#     xis1 += [xi1]*t_n
#     xis2 += [xi2]*t_n*2
    
# print(len(xis1))
# print(len(xis2))
    
# data1['xi'] = xis1 + xis1
# data2['xi'] = xis2 + xis2

d_sets = [data1, data2, data1_err, data2_err]
for d in d_sets:
    d.index = d.index.droplevel([0,1,3,4])
data1.head()

fig, ax = plt.subplots()

data1.xs(level='approx', key=1)[['x']].plot(style='r-', ax=ax)
data2.xs(level='approx', key=1)[['x']].plot(style='b-', ax=ax)

data1.xs(level='approx', key=0)[['x']].plot(style='r-', ax=ax, alpha=0.4)
data2.xs(level='approx', key=0)[['x']].plot(style='b-', ax=ax, alpha=0.4)

axb = ax.twinx()

data1.xs(level='approx', key=1)[['xi']].plot(style='r-', ax=axb)
data2.xs(level='approx', key=1)[['xi']].plot(style='b-', ax=axb)

def plt_err(dta, dtb, obs, ax, color='k'):
    t = dta.xs(level='approx', key=0).index.values
    x = dta.xs(level='approx', key=0)[[obs]].T.values[0]
    dx = dtb.xs(level='approx', key=0)[[obs]].T.values[0]
    ax.fill_between(t, x+dx, x-dx, alpha=0.2, color=color)

plt_err(data1, data1_err, 'x', ax)
plt_err(data2, data2_err, 'x', ax)

In [ ]:
fig, ax = plt.subplots()

data1.xs(level='approx', key=1).groupby('xi').mean()[['x']].plot(style='b--', ax=ax)
data2.xs(level='approx', key=1).groupby('xi').mean()[['x']].plot(style='b--', ax=ax, legend=False)

data1.xs(level='approx', key=0).groupby('xi').mean()[['x']].plot(style='ro', ax=ax, alpha=.3, legend=False)
data2.xs(level='approx', key=0).groupby('xi').mean()[['x']].plot(style='ro', ax=ax, alpha=.3)

dta = data1.groupby(['xi', 'approx']).mean()
dtb = data1_err.groupby(['xi', 'approx']).mean()
plt_err(dta, dtb, 'x', ax, color='k')

dta = data2.groupby(['xi', 'approx']).mean()
dtb = data2_err.groupby(['xi', 'approx']).mean()
plt_err(dta, dtb, 'x', ax, color='k')

h,l = ax.get_legend_handles_labels()
ax.legend([h[1], h[3]],["approximation", "micro model"], loc=2)

ax.set_xlim([.115, .14])
dta.head()